# Create test dataset for DNNDK

In [2]:
import tensorflow as tf

In [3]:
import os
import shutil
import glob

In [ ]:
from tqdm import tqdm

In [10]:
tf.enable_eager_execution()

In [4]:
print("TensorFlow version: %s" % tf.__version__)

TensorFlow version: 1.14.0


***

In [5]:
dir_train = "D:\\dataset\\imagenet\\img_test"

In [6]:
output_dir = '_test'

In [15]:
image_list = 'test.txt'

In [16]:
img_size = 224

In [7]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)

In [12]:
# Enumerate sub directories
test_imgs = glob.glob(os.path.join(dir_train, '*.JPEG'), recursive=False)

In [13]:
def calc_new_size(img, smallest_side):
    height = tf.cast(img.shape[0], dtype=tf.float32)  # Height
    width =  tf.cast(img.shape[1], dtype=tf.float32)  # Width
    smallest_side = tf.cast(smallest_side, dtype=tf.float32)
    scale = tf.cond(tf.greater(height, width),
                  lambda: smallest_side / width,
                  lambda: smallest_side / height)

    new_height = tf.multiply(height, scale)
    new_width = tf.multiply(width, scale)

    offset_h = tf.cast(tf.subtract(new_height, smallest_side) / 2, tf.int32)
    offset_w = tf.cast(tf.subtract(new_width, smallest_side) / 2, tf.int32)
    
    new_height = tf.cast(new_height, tf.int32)
    new_width  = tf.cast(new_width, tf.int32)
    
    return new_height, new_width, offset_h, offset_w

In [17]:
bar = tqdm(total = len(calib_imgs))
bar.set_description('Progress')

for img_path in test_imgs:
    with tf.gfile.GFile(img_path, 'rb') as fr:
        img = tf.image.decode_image(fr.read())

        # Calculate Offset & size after resize
        new_height, new_width, offset_h, offset_w = calc_new_size(img, img_size + 1)

        # expand_dims(): Input to resize_bilinear must be a 4-D tensor
        img_resize = tf.image.resize_bilinear(
            tf.expand_dims(img, 0), [new_height, new_width], align_corners=False)

        # Crop
        img2 = tf.image.crop_to_bounding_box(
            img_resize, offset_h, offset_w, img_size, img_size)

        # Save as .png
        _, name = os.path.split(img_path)
        name = os.path.splitext(name)[0]

        png_data = tf.image.encode_png(tf.cast(img2[0], dtype=tf.uint8))
        with tf.gfile.GFile(os.path.join(output_dir, name + '.png'), 'wb') as fw:
            fw.write(png_data.numpy())
    
    # Update progress bar
    bar.update(1)

In [ ]:
# Save image list
open(os.path.join(output_dir, image_list), 'w') as f_list:
    for img_path in test_imgs:
        _, name = os.path.split(img_path)
        name = os.path.splitext(name)[0]
        f_list.write(output_dir+'/'+name+'.png'+'\n')